In [7]:
#先從資料庫抓資料出來
from crossref.restful import Journals
import pandas as pd
def data_api(issn,journal_name):
    authors_list = []
    title_list = []
    affiliations_list = []
    published_dates_list=[]
    journal_names_list=[]
    subjects_list=[]
    volumes_list=[]
    issues_list=[]
    URL_list=[]
    DOI_list=[]
    count = 0
    journals = Journals()
    results = journals.works(issn=issn)
    search = results.filter(from_pub_date="2000").select(["container-title","author","DOI","URL",'subject','volume',"title",'issue',"published-print"])
    total = search.count()
    for item in search:
        authors = [f"{author.get('given', '')} {author.get('family', '')}" for author in item.get('author', [])]
        count += 1
        if len(authors) >0:
            authors_list.append('; '.join(authors))
            # Get the affiliations
            affiliations = [aff.get('name', '') for author in item.get('author', []) for aff in author.get('affiliation', [])]
            affiliations_list.append('; '.join(affiliations))
            title_list.append(item.get('title', '')[0])
            published_date = '-'.join(str(part) for part in item.get('published-print', {}).get('date-parts', [[]])[0])
            published_dates_list.append(published_date)
            journal_names_list.append(item.get('container-title', [''])[0])
            subjects_list.append('; '.join(item.get('subject', [])))
            volumes_list.append(item.get('volume', ''))
            issues_list.append(item.get('issue', ''))
            URL_list.append(item.get('URL',''))
            DOI_list.append(item.get('DOI',''))
            print("load success",count,"/",total)
    data = {
            'Authors': authors_list,
            'Title' : title_list,
            'Affiliations': affiliations_list,
            'Published Date': published_dates_list,
            'Journal Name ': journal_names_list, 
            'Subjects': subjects_list,
            'Volume': volumes_list,
            'Issue': issues_list,
            'DOI': DOI_list,
            'URL': URL_list
    
        }
    df = pd.DataFrame(data)
        
    # Save the DataFrame to a CSV file
    df.to_csv(f'{journal_name}_api.csv', index=True)

In [2]:
#formal
#到網頁上補資料
import cloudscraper
import pandas as pd
import random
import time
from bs4 import BeautifulSoup
def data_check(journal_name,redo = False,start = 0):
    error_occur = False
    # create cloudscraper
    scraper = cloudscraper.create_scraper()
    if redo == False:
        df = pd.read_csv(f"{journal_name}_api.csv", index_col=0)
    else:
        df = pd.read_csv(f"{journal_name}.csv", index_col=0)
    count = start
    total = len(df["DOI"])
    
    for index, row in df.iloc[start:].iterrows():
        url = row["URL"]
        print(url)
        affiliation = row["Affiliations"]
    
        # check affiliation 
        if pd.isna(affiliation) or df.iloc[index]["Title"] == df.iloc[index-1]["Title"]:
            print("open")
            try:
                # to URL
                response = scraper.get(url)
                random_wait_object = random.uniform(1,100)
                if count%100 == random_wait_object:
                    random_wait_time = random.uniform(60,80)
                elif count%400 == 0 and count != 0:
                    random_wait_time = random.uniform(600,1000)
                else:
                    random_wait_time = random.uniform(5,12)
                time.sleep(random_wait_time)
                if response.status_code == 403:
                    print(403,"error occur",url,count,"/",total)
                    df.to_csv(f'{journal_name}.csv',index=True)
                    time.sleep(1000)
                    data_check(journal_name,redo = True,start = index)
                    error_occur = True
                    break
                if response.status_code == 200:
                    print("open success")
                    # get html
                    page_source = response.text
                
                    # parse html by beatifulsoup
                    soup = BeautifulSoup(page_source, 'html.parser')
                
                    #get author
                    authors = [meta['content'] for meta in soup.find_all('meta', {'name': 'citation_author'})]
                
                    # get citation_author_institution
                    author_institutions = [meta['content'] for meta in soup.find_all('meta', {'name': 'moreInfoLink'})]
                
                    # get citation_publication_date
                    publication_date = soup.find('meta', {'name': 'citation_publication_date'})['content'] if soup.find('meta', {'name': 'citation_publication_date'}) else None
                
                    # get citation_title
                    title = soup.find('meta', {'name': 'citation_title'})['content'] if soup.find('meta', {'name': 'citation_title'}) else None
                    authors_str = '; '.join(authors)
                    author_institutions_str = '; '.join(author_institutions)
                    # update df
                    df.at[index, 'Authors'] = authors_str
                    df.at[index, 'Affiliations'] = author_institutions_str
                    df.at[index, 'Published Date'] = publication_date
                    df.at[index, 'Title'] = title
                    print(df.iloc[index])
                    print('success',count,"/",total)
                else:
                    print(f"Failed to retrieve: {response.status_code}",count,"/",total)
            except Exception as e:
                print(f"An error occurred: {e}",count,"/",total)
            finally:
                count += 1
        else:
            count += 1
            print(f"Skipping {url} as Affiliations is not empty.",count,"/",total)
            
    # save CSV
    if error_occur == False:
        df.to_csv(f'{journal_name}.csv',index=True)


In [5]:
#formal
import cloudscraper
import pandas as pd
import random
import time
from bs4 import BeautifulSoup
def data_check_by_div(journal_name,redo = False,start = 0):
    keywords = ['university', 'institution', 'academia sinica','school']
    error_occur = False
    # create cloudscraper
    scraper = cloudscraper.create_scraper()
    if redo == False:
        df = pd.read_csv(f"{journal_name}_api.csv", index_col=0)
    else:
        df = pd.read_csv(f"{journal_name}.csv", index_col=0)
    count = start+1
    total = len(df["DOI"])+1
    
    for index, row in df.iloc[start:].iterrows():
        url = "https://onlinelibrary.wiley.com/doi/abs/"+row["DOI"]
        print(url)
        affiliation = row["Affiliations"]
    
        # check affiliation
        if pd.isna(affiliation) or df.iloc[index]["Title"] == df.iloc[index-1]["Title"]:
            print("open")
            try:
                # to URL
                response = scraper.get(url)
                random_wait_object = random.uniform(1,100)
                if count%100 == random_wait_object:
                    random_wait_time = random.uniform(60,80)
                elif count%400 == 0 :
                    random_wait_time = random.uniform(600,1000)
                else:
                    random_wait_time = random.uniform(5,12)
                time.sleep(random_wait_time)
                if response.status_code == 403:
                    print(403,"error occur",index,url)
                    df.to_csv(f'{journal_name}.csv',index=True)
                    time.sleep(1000)
                    data_check(journal_name,redo = True,start = index)
                    error_occur = True
                    break
                if response.status_code == 200:
                    print("open success")
                    # get html
                    page_source = response.text
                
                    # parse html 
                    soup = BeautifulSoup(page_source, 'html.parser')
                    #get author
                    desktop_authors_div = soup.find('div', class_='loa-wrapper loa-authors hidden-xs desktop-authors')
                    if desktop_authors_div:
                        # find all author_div
                        author_info_divs = desktop_authors_div.find_all('div', class_='author-info accordion-tabbed__content')
                    
                        # empty list to store affiliation information
                        author_institutions = []
                    
                        # go through author-info div
                        for div in author_info_divs:
                            if any(keyword.lower() in div.text.lower() for keyword in keywords):
                                            # if containing keywords, save affiliation
                                    author_institutions.append(div.text)
                    
                    else:
                        print(f"No desktop_authors_div found for URL: {url}")
                        author_institutions =[]
                
                    # get author
                    #authors = [meta['content'] for meta in soup.find_all('meta', {'name': 'citation_author'})]
                
                    # get citation_author_institution
                    #author_institutions = [meta['content'] for meta in soup.find_all('meta', {'name': 'moreInfoLink'})]
                
                    # get  citation_publication_date
                    publication_date = soup.find('meta', {'name': 'citation_publication_date'})['content'] if soup.find('meta', {'name': 'citation_publication_date'}) else None
                
                    # get citation_title
                    title = soup.find('meta', {'name': 'citation_title'})['content'] if soup.find('meta', {'name': 'citation_title'}) else None
                    #authors_str = '; '.join(authors)
                    author_institutions_str = '; '.join(author_institutions)
                    # update df
                    #df.at[index, 'Authors'] = authors_str
                    df.at[index, 'Affiliations'] = author_institutions_str
                    df.at[index, 'Published Date'] = publication_date
                    df.at[index, 'Title'] = title
                    print(df.iloc[index])
                    print('success',count,"/",total)
                else:
                    print(f"Failed to retrieve: {response.status_code}",count,"/",total)
            except Exception as e:
                print(f"An error occurred: {e}",count,"/",total)
            finally:
                count += 1
        else:
            count += 1
            print(f"Skipping {url} as Affiliations is not empty.",count,"/",total)
            
    # save CSV
    if error_occur == False:
        df.to_csv(f'{journal_name}.csv',index=True)

In [6]:
import pandas as pd
def taiwan_filter(journal_name):
    taiwan_universities = [
    "National Taiwan University",
    "National Tsing Hua University",
    "National Chiao Tung University",
    "National Cheng Kung University",
    "Taipei Medical University",
    "National Yang Ming University",
    "Chung Yuan Christian University",
    "National Central University",
    "National Sun Yat-sen University",
    "National Taiwan Normal University",
    "National Chung Cheng University",
    "National Taitung University",
    "National Chi Nan University",
    "Tunghai University",
    "National Chengchi University",
    "Soochow University",
    "Chang Gung University",
    "Providence University",
    "Feng Chia University",
    "Tamkang University",
    "I-Shou University",
    "National Taipei University of Technology",
    "Aletheia University",
    "St. John's University",
    "Nanhua University",
    "Chia Nan University of Pharmacy & Science",
    "Tung Fang Design University",
    "Asia University",
    "Ming Chuan University",
    "Kainan University",
    "Chien Hsin University of Science & Technology",
    "Southern Taiwan University of Science & Technology",
    "Takming University of Science & Technology",
    "National Taipei University",
    "Chinese Culture University",
    "National Dong Hwa University",
    "Shih Hsin University",
    "Yuan Ze University",
    "Tzu Chi University",
    "Hsuan Chuang University",
    "Tatung University",
    "National Taiwan Ocean University",
    "National Taipei University of Education",
    "National Ilan University",
    "Chaoyang University of Technology",
    "National Formosa University",
    "Da-Yeh University",
    "Chung Hua University",
    "Far East University",
    "Chung Jen University",
    "Shu-Te University",
    "Shih Chien University",
    "Ta Hwa University",
    "De Lin Institute of Technology",
    "Vanung University",
    "Tungnan University",
    "China Medical University",
    "Fu Jen Catholic University",
    "Chung Yuan Christian University",
    "Chihlee University of Technology",
    "Ming Chuan University",
    "University of Taipei",
    "Kainan University",
    "Tatung University",
    "Chang Gung University",
    "Tzu Chi University",
    "Nanhua University",
    "Tung Fang Design University",
    "Chien Hsin University of Science & Technology",
    "Chaoyang University of Technology",
    "Da-Yeh University",
    "Chung Hua University",
    "Far East University",
    "Chung Jen University",
    "Shu-Te University",
    "Shih Chien University",
    "Ta Hwa University",
    "De Lin Institute of Technology",
    "Vanung University",
    "Tungnan University",
    "China Medical University",
    "Fu Jen Catholic University",
    "Chung Yuan Christian University",
    "Chihlee University of Technology",
    "Ming Chuan University",
    "University of Taipei",
    "Kainan University",
    "Tatung University",
    "Chang Gung University",
    "Tzu Chi University",
    "Nanhua University",
    "Tung Fang Design University",
    "Chien Hsin University of Science & Technology",
    "Academia Sinica",
    "Taiwan"
    'taiwan'
    ".tw"
    ]
    
    journal = pd.read_csv(f"{journal_name}.csv")
    print("open success")
    # 创建一个条件以判断"Institutions"列是否包含台湾排名前四十大学、"Academia Sinica"和"Jinan University"
    has_taiwan_institutions = journal["Affiliations"].str.contains("|".join(taiwan_universities), case=False, na=False)
    
    # 筛选包含台湾排名前四十大学、"Academia Sinica"和"Jinan University"的行
    data_taiwan = journal[has_taiwan_institutions]
    
    # 保存包含台湾排名前四十大学、"Academia Sinica"和"Jinan University"的行为一个新的CSV文件
    file_taiwan = f"{journal_name}_taiwan_scholar.csv"
    data_taiwan.to_csv(file_taiwan, index=False)
    
    print(f"已提取包含台湾排名前四十大学、Academia Sinica 和 Jinan University 的数据并保存为 {file_taiwan}")

In [22]:
journal_list = ['Economics Journal', 'Econometrica' ,'Journal of Finance', 'Quantitative Economics','Theoretical Economics']
for journal_name in journal_list:
    taiwan_filter(journal_name)

已提取包含台湾排名前四十大学、Academia Sinica 和 Jinan University 的数据并保存为 Economics Journal_taiwan_scholar.csv
已提取包含台湾排名前四十大学、Academia Sinica 和 Jinan University 的数据并保存为 Econometrica_taiwan_scholar.csv
已提取包含台湾排名前四十大学、Academia Sinica 和 Jinan University 的数据并保存为 Journal of Finance_taiwan_scholar.csv
已提取包含台湾排名前四十大学、Academia Sinica 和 Jinan University 的数据并保存为 Quantitative Economics_taiwan_scholar.csv
已提取包含台湾排名前四十大学、Academia Sinica 和 Jinan University 的数据并保存为 Theoretical Economics_taiwan_scholar.csv


In [8]:
data_api('0012-9682','Econometrica_test')

load success 8 / 2308
load success 11 / 2308
load success 39 / 2308
load success 47 / 2308
load success 48 / 2308
load success 51 / 2308
load success 80 / 2308
load success 81 / 2308
load success 84 / 2308
load success 91 / 2308
load success 94 / 2308
load success 95 / 2308
load success 98 / 2308
load success 109 / 2308
load success 113 / 2308
load success 114 / 2308
load success 120 / 2308
load success 123 / 2308
load success 129 / 2308
load success 137 / 2308
load success 138 / 2308
load success 139 / 2308
load success 140 / 2308
load success 141 / 2308
load success 142 / 2308
load success 144 / 2308
load success 148 / 2308
load success 149 / 2308
load success 152 / 2308
load success 159 / 2308
load success 160 / 2308
load success 161 / 2308
load success 163 / 2308
load success 164 / 2308
load success 165 / 2308
load success 166 / 2308
load success 167 / 2308
load success 168 / 2308
load success 169 / 2308
load success 171 / 2308
load success 172 / 2308
load success 175 / 2308
load s

In [27]:
taiwan_filter('Econometrica_test_api')

已提取包含台湾排名前四十大学、Academia Sinica 和 Jinan University 的数据并保存为 Econometrica_test_api_taiwan_scholar.csv


In [ ]:
data_check('Econometrica_test',redo=False,start=0)

http://dx.doi.org/10.3982/ecta16729
Skipping http://dx.doi.org/10.3982/ecta16729 as Affiliations is not empty. 1 / 1201
http://dx.doi.org/10.1111/j.1468-0262.2005.00602.x
open
403 error occur http://dx.doi.org/10.1111/j.1468-0262.2005.00602.x 1 / 1201


In [76]:
data_check_by_div('Journal of Finance',redo=False,start=0)

https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1540-6261.2007.01242.x
open
open success
Authors                                                    BING HAN
Title             Stochastic Volatilities and Correlations of Bo...
Affiliations      \nBING HANHan is from the McCombs School of Bu...
Published Date                                           2007/06/01
Journal Name                                 The Journal of Finance
Subjects            Economics and Econometrics; Finance; Accounting
Volume                                                         62.0
Issue                                                           3.0
DOI                                10.1111/j.1540-6261.2007.01242.x
URL               http://dx.doi.org/10.1111/j.1540-6261.2007.012...
Name: 0, dtype: object
success 1 / 1889
https://onlinelibrary.wiley.com/doi/abs/10.1111/jofi.13273
open
open success
Authors                                          LINDA M. SCHILLING
Title                        Optimal Forbear

In [51]:
redo_list = ["Econometrica","Theoretical Economics"]
for name in redo_list:
    data_check(name,redo = True,start=0)

https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1468-0262.2005.00602.x
Skipping https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1468-0262.2005.00602.x as Affiliations is not empty. 1 / 548
https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1468-0262.2004.00533.x
open
open success
Authors                                        Xueli Tang; Ian King
Title             A Comment on Roland Benabou's “Tax and Educati...
Affiliations      1Dept. of Economics, University of Auckland, 3...
Published Date                                           2005/05/01
Journal Name                                           Econometrica
Subjects                                 Economics and Econometrics
Volume                                                           73
Issue                                                             3
DOI                                10.1111/j.1468-0262.2005.00602.x
URL               http://dx.doi.org/10.1111/j.1468-0262.2005.006...
Name: 0, dtype: object
success 1 / 5

In [3]:
#單個爬蟲
#要先到網站上看網站結構才知道要怎麼改程式
def data_scraper(scraper,url):
  response = scraper.get(url)
  if response.status_code == 200:
    random_wait_time = random.uniform(5,12)
    time.sleep(random_wait_time)
    page_source = response.text
    soup = BeautifulSoup(page_source, 'html.parser')
    print(soup)
  else:
    print("fail to open")
  return(response.status_code)
  

In [4]:
import cloudscraper
import pandas as pd
import random
import time
from bs4 import BeautifulSoup
import urllib.parse
scraper = cloudscraper.create_scraper()
url = "http://dx.doi.org/10.1016/j.jfineco.2010.06.007"
data_scraper(scraper,url)
#yeest



<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "https://www.w3.org/TR/html4/loose.dtd">

<html>
<head>
<meta charset="utf-8"/>
<meta content="text/html" http-equiv="Content-Type"/>
<meta content="2; url='/retrieve/articleSelectSinglePerm?Redirect=https%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS0304405X1000156X%3Fvia%253Dihub&amp;key=ee7388e823d41165a2e0c0d8e59e406a1b728309'" http-equiv="REFRESH">
<script src="/retrieve/static/script/siteCatalystHelper.js" type="text/JavaScript"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.10.1/jquery.min.js" type="text/JavaScript"></script>
<script src="https://assets.adobedtm.com/4a848ae9611a/519188bfaff2/launch-d9c9ae460653.min.js"></script>
<title>Redirecting</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script language="JavaScript" type="text/javascript">
var timerStart = Date.now();
var pageName = "";

if (pageName == "") {
	pageName = "Auto Article Locator"

200

In [15]:
import cloudscraper
import pandas as pd
import random
import time
from bs4 import BeautifulSoup
import urllib.parse
def data_scraper_redirect(scraper,url):
  response = scraper.get(url)
  if response.status_code == 200:
    global new_soup
    random_wait_time = random.uniform(5,12)
    time.sleep(random_wait_time)
    page_source = response.text
    soup = BeautifulSoup(page_source, 'html.parser')
    encoded_url = soup.find('input', {'id': 'redirectURL'})['value']
    redirect_url = urllib.parse.unquote(encoded_url)
    random_wait_time = random.uniform(5,12)
    time.sleep(random_wait_time)
    response = scraper.get(redirect_url)
    new_html = response.text
    new_soup = BeautifulSoup(new_html, 'html.parser')
    print(new_soup)
  else:
    print("fail to open")
  return(response.status_code)

In [17]:
scraper = cloudscraper.create_scraper()
url = "http://dx.doi.org/10.1016/j.jfineco.2021.11.004"
data_scraper_redirect(scraper,url)

<!DOCTYPE html>

<!--[if IE 9]><html class="ie9" lang={defaultLocale}><![endif]-->
<html lang="en-US">
<head>
<meta content="S0304405X21004931" name="citation_pii"/>
<meta content="0304-405X" name="citation_issn"/>
<meta content="146" name="citation_volume"/>
<meta content="778" name="citation_lastpage"/>
<meta content="2" name="citation_issue"/>
<meta content="North-Holland" name="citation_publisher"/>
<meta content="754" name="citation_firstpage"/>
<meta content="Journal of Financial Economics" name="citation_journal_title"/>
<meta content="JOUR" name="citation_type"/>
<meta content="10.1016/j.jfineco.2021.11.004" name="citation_doi"/>
<meta content="10.1016/j.jfineco.2021.11.004" name="dc.identifier"/>
<meta content="Full-length article" name="citation_article_type"/>
<meta content="We show that negative interest rate policy (NIRP) has expansionary effects on credit supply through a portfolio rebalancing channel. By shifting down …" property="og:description"/>
<meta content="https:/

200

In [8]:
data_api('0304-405X','Journal of Financial Economics')

load success 2 / 2970
load success 7 / 2970
load success 8 / 2970
load success 9 / 2970
load success 12 / 2970
load success 16 / 2970
load success 17 / 2970
load success 18 / 2970
load success 25 / 2970
load success 26 / 2970
load success 33 / 2970
load success 34 / 2970
load success 35 / 2970
load success 36 / 2970
load success 38 / 2970
load success 43 / 2970
load success 44 / 2970
load success 45 / 2970
load success 46 / 2970
load success 47 / 2970
load success 50 / 2970
load success 51 / 2970
load success 52 / 2970
load success 53 / 2970
load success 54 / 2970
load success 56 / 2970
load success 57 / 2970
load success 58 / 2970
load success 63 / 2970
load success 64 / 2970
load success 65 / 2970
load success 66 / 2970
load success 67 / 2970
load success 68 / 2970
load success 72 / 2970
load success 73 / 2970
load success 74 / 2970
load success 75 / 2970
load success 76 / 2970
load success 81 / 2970
load success 85 / 2970
load success 86 / 2970
load success 87 / 2970
load success 93

In [79]:
#更改後
import cloudscraper
import pandas as pd
import random
import time
from bs4 import BeautifulSoup
import urllib.parse
import json

def data_scraper_redirect(scraper, url):
    response = scraper.get(url)
    if response.status_code == 200:
        random_wait_time = random.uniform(5, 12)
        time.sleep(random_wait_time)
        page_source = response.text
        soup = BeautifulSoup(page_source, 'html.parser')
        encoded_url = soup.find('input', {'id': 'redirectURL'})['value']
        redirect_url = urllib.parse.unquote(encoded_url)
        random_wait_time = random.uniform(1,3)
        time.sleep(random_wait_time)
        response = scraper.get(redirect_url)
        new_html = response.text
        new_soup = BeautifulSoup(new_html, 'html.parser')
    else:
        print("fail to open")
        new_soup = None
    return new_soup

def data_check(journal_name, redo=False, start=0):
    #store information
    columns = ['Author', 'Affiliation', 'Publication Date', 'Journal Title', 'Title', 'Volume', 'Issue', 'URL']
    result_df = pd.DataFrame(columns=columns)
    #affiliation key words
    key_words = ["University","Bank","Institution","Sinica","Department","Finance","Economics","Federal","College","Fund"]
    #record error
    error_occur = False
    #create scraper
    scraper = cloudscraper.create_scraper()
    if redo == False:
        df = pd.read_csv(f"{journal_name}_api.csv", index_col=0)
    else:
        df = pd.read_csv(f"{journal_name}.csv", index_col=0)
    count = start + 1
    total = len(df["DOI"])

    for index, row in df.iloc[start:].iterrows():
        url = row["URL"]
        print(url)
        affiliation = row["Affiliations"]
        affiliation_list = []
        authors_list = []
        if pd.isna(affiliation) or df.iloc[index]["Title"] == df.iloc[index - 1]["Title"]:
            print("open")
            try:
                # Call your new function here to handle the redirect
                new_soup = data_scraper_redirect(scraper, url)
                if new_soup:
                    script_tags = new_soup.find_all('script', type='application/json', attrs={"data-iso-key": "_0"})
                    if script_tags:
                        script_tag = script_tags[0]  # Get the first script tag that matches the criteria
                        json_text = script_tag.string or script_tag.text  # Get the text content of the script tag
                        json_data = json.loads(json_text)  # Parse the JSON data
                        # Now you can access the JSON data as a dictionary
                        #Affiliation
                        authors_content = json_data.get('authors', {}).get('content', [])
                        for author_info in authors_content:
                            for affiliation_info in author_info.get('$$', []):
                                if affiliation_info.get('#name') == 'affiliation':
                                    if affiliation_info:
                                        affiliation_text = affiliation_info.get('$$', [{}])[0].get("_")
                                        contains_key_word = any(key_word in affiliation_text for key_word in key_words)
                                        if contains_key_word:
                                            affiliation_list.append(affiliation_text)
                                        else:
                                            affiliation_text = affiliation_info.get('$$', [{}])[1].get("_")
                                            affiliation_list.append(affiliation_text)
                        #author name
                        # 遍歷content列表
                        for author_group in json_data['authors'].get('content', []):
                            # 遍歷$$列表
                            for author_data in author_group.get('$$', []):
                                # 檢查當前項是否是一個作者項
                                if author_data.get('#name') == 'author':
                                    # 提取作者的名字和姓氏
                                    author_info = {}
                                    for field in author_data.get('$$', []):
                                        if field.get('#name') in ['given-name', 'surname']:
                                            field_name = field.get('#name')
                                            field_value = field.get('_')
                                            author_info[field_name] = field_value
                                    # 將作者資訊添加到作者列表中
                                    authors_list.append(author_info)
                        
                #other information
                volume = new_soup.find('meta', {'name': 'citation_volume'})['content']
                issue = new_soup.find('meta', {'name': 'citation_issue'})['content']
                journal_title = new_soup.find('meta', {'name': 'citation_journal_title'})['content']
                title = new_soup.find('meta', {'name': 'citation_title'})['content']
                publication_date = new_soup.find('meta', {'name': 'citation_publication_date'})['content']                        
                #integrate
                authors_str = '; '.join(f"{author['given-name']} {author['surname']}" for author in authors_list)
                author_institutions_str = '; '.join(affiliation_list)
                # update df
                result_df.loc[index] = [authors_str, author_institutions_str, publication_date, journal_title, title, volume, issue, url]
                print(result_df.iloc[index])
                print('success',count,"/",total)
                                        
            except Exception as e:
                print(f"An error occurred: {e}", count, "/", total)
            finally:
                count += 1
        else:
            count += 1
            print(f"Skipping {url} as Affiliations is not empty.", count, "/", total)

    if error_occur == False:
        result_df['Volume'] = result_df['Volume'].astype(int)
        result_df.sort_values(by='Volume', ascending=False, inplace=True)
        result_df.to_csv(f'{journal_name}.csv', index=False)


In [80]:
#再試一次
data_check('Journal of Financial Economics',redo=False,start=0)

http://dx.doi.org/10.1016/j.jfineco.2010.06.007
open
3
Author                                     Paul Schultz; Sophie Shive
Affiliation         Department of Finance, Mendoza College of Busi...
Publication Date                                           2010/12/01
Journal Title                          Journal of Financial Economics
Title               Mispricing of dual-class shares: Profit opport...
Volume                                                             98
Issue                                                               3
URL                   http://dx.doi.org/10.1016/j.jfineco.2010.06.007
Name: 0, dtype: object
success 1 / 2669
http://dx.doi.org/10.1016/j.jfineco.2021.11.004
open
3
3
3
3
3
3
Author              Margherita Bottero; Camelia Minoiu; José-Luis ...
Affiliation         Bank of Italy, Italy; Federal Reserve Board, U...
Publication Date                                           2022/11/01
Journal Title                          Journal of Financial Economics


KeyboardInterrupt: 

In [25]:
import re
import json
from bs4 import BeautifulSoup

# 假设您已经有了一个BeautifulSoup对象
# soup = BeautifulSoup(html_content, 'html.parser')

script_tags = new_soup.find_all('script')
for tag in script_tags:
    script_content = tag.string
    if script_content:
        match = re.search(r'var someVariableName = ({.*?});', script_content)
        if match:
            json_str = match.group(1)
            json_content = json.loads(json_str)
            if 'affiliation' in json_content:
                affiliation_info = json_content['affiliation']
                print("success",affiliation_info)

In [58]:
script_tags = new_soup.find_all('script', type='application/json', attrs={"data-iso-key": "_0"})
key_words = ["University","Bank","Institution","Sinica","Department","Finance","Economics","Federal",'fund']
import re
import json
from bs4 import BeautifulSoup
if script_tags:
    script_tag = script_tags[0]  # Get the first script tag that matches the criteria
    json_text = script_tag.string or script_tag.text  # Get the text content of the script tag
    json_data = json.loads(json_text)  # Parse the JSON data

    # Now you can access the JSON data as a dictionary
    # affiliation
    authors_content = json_data.get('authors', {}).get('content', [])
    for author_info in authors_content:
        for affiliation_info in author_info.get('$$', []):
            if affiliation_info.get('#name') == 'affiliation':
                if affiliation_info:
                    affiliation_text = affiliation_info.get('$$', [{}])[1].get("_")
                    contains_key_word = any(key_word in affiliation_text for key_word in key_words)
                    if contains_key_word:
                        pass
    #author name
    authors_list = []
    # 遍歷content列表
    for author_group in json_data['authors'].get('content', []):
        # 遍歷$$列表
        for author_data in author_group.get('$$', []):
            # 檢查當前項是否是一個作者項
            if author_data.get('#name') == 'author':
                # 提取作者的名字和姓氏
                author_info = {}
                for field in author_data.get('$$', []):
                    if field.get('#name') in ['given-name', 'surname']:
                        field_name = field.get('#name')
                        field_value = field.get('_')
                        author_info[field_name] = field_value
                # 將作者資訊添加到作者列表中
                authors_list.append(author_info)
    authors_list = '; '.join(f"{author['given-name']} {author['surname']}" for author in authors_list)
    
    # 輸出作者列表
    print(authors_list)
    #other information
    volume = new_soup.find('meta', {'name': 'citation_volume'})['content']
    issue = new_soup.find('meta', {'name': 'citation_issue'})['content']
    journal_title = new_soup.find('meta', {'name': 'citation_journal_title'})['content']
    title = new_soup.find('meta', {'name': 'citation_title'})['content']
    publication_date = new_soup.find('meta', {'name': 'citation_publication_date'})['content']
    
    # 輸出結果
    print(f'Volume: {volume}')
    print(f'Issue: {issue}')
    print(f'Journal Title: {journal_title}')
    print(f'Title: {title}')
    print(f'Publication Date: {publication_date}')
        
    

            

Margherita Bottero; Camelia Minoiu; José-Luis Peydró; Andrea Polo; Andrea F. Presbitero; Enrico Sette
Volume: 146
Issue: 2
Journal Title: Journal of Financial Economics
Title: Expansionary yet different: Credit supply and real effects of negative interest rate policy
Publication Date: 2022/11/01


In [47]:
print(new_soup)

<!DOCTYPE html>

<!--[if IE 9]><html class="ie9" lang={defaultLocale}><![endif]-->
<html lang="en-US">
<head>
<meta content="S0304405X21004931" name="citation_pii"/>
<meta content="0304-405X" name="citation_issn"/>
<meta content="146" name="citation_volume"/>
<meta content="778" name="citation_lastpage"/>
<meta content="2" name="citation_issue"/>
<meta content="North-Holland" name="citation_publisher"/>
<meta content="754" name="citation_firstpage"/>
<meta content="Journal of Financial Economics" name="citation_journal_title"/>
<meta content="JOUR" name="citation_type"/>
<meta content="10.1016/j.jfineco.2021.11.004" name="citation_doi"/>
<meta content="10.1016/j.jfineco.2021.11.004" name="dc.identifier"/>
<meta content="Full-length article" name="citation_article_type"/>
<meta content="We show that negative interest rate policy (NIRP) has expansionary effects on credit supply through a portfolio rebalancing channel. By shifting down …" property="og:description"/>
<meta content="https:/

In [41]:
script_tags

[<script data-iso-key="_0" type="application/json">{"abstracts":{"content":[{"$$":[{"$":{"id":"sctt0001"},"#name":"section-title","_":"Abstract"},{"$$":[{"$":{"view":"all","id":"sp0001"},"#name":"simple-para","_":"We show that negative interest rate policy (NIRP) has expansionary effects on credit supply through a portfolio rebalancing channel. By shifting down and flattening the yield curve, NIRP differs from rate cuts just above the zero-lower-bound and has effects similar to QE. For identification, we exploit ECB’s NIRP and the Italian credit register and, for external validity, European and U.S. datasets. NIRP affects more banks with higher ex-ante liquid assets, including net interbank positions. More exposed banks reduce liquid assets, expand credit supply, especially to financially-constrained firms, and cut loan rates, inducing firms to increase investment and the wage bill."}],"$":{"view":"all","id":"abssec0001"},"#name":"abstract-sec"}],"$":{"view":"all","id":"abs0001","class